In [1]:
# Importamos librerías de Aprendizaje automático
from sklearn import preprocessing
from sklearn import linear_model
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.svm import SVR, LinearSVR
from sklearn.metrics import make_scorer
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectFromModel
from sklearn.decomposition import PCA
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.feature_selection import VarianceThreshold


In [2]:
# ahora importamos el dataframe que procesamos en el EDA
import pandas as pd
import numpy as np
airbnb_df = pd.read_csv("airbnb_df.csv")

In [3]:
# ahora procedemos a dividir el dataframe entre variables independientes 'X' y dependientes 'Y'
y = airbnb_df['price']
x = airbnb_df.drop(columns=['price'])

In [4]:
# usamos la librería de sklearn y determinamos el tamaño del conjunto de testeo en 20%
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0)

In [5]:
# primero ejecutaremos sin reduccion de dimensionalidad en el pipeline1 o 'PL1'
# al finalizar, haremos otra prediccion en el pipeline2 o 'PL2' esta vez reduciendo la dimensionalidad
# usaremos la funcion 'Pipeline' de sklearn para ordenar la ejecucion del codigo

PL1 = Pipeline(steps=[
    ('scaler', StandardScaler()),
    ('model', LinearRegression())
])
# una vez escalados los datos y habiendo definido el modelo, lo entrenamos
PL1.fit(x_train, y_train)

,steps,"[('scaler', ...), ('model', ...)]"
,transform_input,None
,memory,None
,verbose,False
,copy,True
,with_mean,True
,with_std,True
,fit_intercept,True
,copy_X,True
,tol,1e-06
,n_jobs,None


In [6]:
# para el PL2 vamos a probar otro modelo más complejo con la esperanza de que mejore el score del PL1
# al tener tantas dummies para las var categoricas, la dimensionalidad del df es elevada y seguramente requiere una reduccion.

# ahora ejecutaremos el PL2 utilizando la tecnica de Gradient Boosting con reduccion de dimensionalidad PCA
# en el scaler usamos with_mean=False por las variables dummies que explican las variables categoricas
# antes del entrenamiento usamos threshold 0.01 para eliminar las variables casi nulas que no son relevantes. esto va a agilizar el proceso
# para la reduccion fijamos los componentes a conciderar en 100 para capturar una buena porcion de la varianza de la variable 'price'
# fijamos el nuemro de arboles en 100 con n_estimators y una complejidad de 3 niveles con max_depth
PL2 = Pipeline([
    ('scaler', StandardScaler(with_mean=False)),
    ('var_thresh', VarianceThreshold(threshold=0.01)),
    ('reduccion', PCA(n_components=100)),
    ('modelo', GradientBoostingRegressor(random_state=0))
])
param_grid = {
    "modelo__n_estimators": [100],
    "modelo__learning_rate": [0.05, 0.1],
    "modelo__max_depth": [3]
}

grid_PL2 = GridSearchCV(
    PL2,
    param_grid,
    cv=5,
    scoring="neg_root_mean_squared_error",
    n_jobs=-1
)
grid_PL2.fit(x_train, y_train)

,estimator,Pipeline(step...om_state=0))])
,param_grid,"{'modelo__learning_rate': [0.05, 0.1], 'modelo__max_depth': [3], 'modelo__n_estimators': [100]}"
,scoring,'neg_root_mean_squared_error'
,n_jobs,-1
,refit,True
,cv,5
,verbose,0
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,False
,copy,True


In [7]:
print("Mejores parámetros encontrado con el modelo de PL2:", grid_PL2.best_params_)
# guardamos en una variable el mejor modelo resultado del gridsearch en PL2
PL2_optimo = grid_PL2.best_estimator_

Mejores parámetros encontrado con el modelo de PL2: {'modelo__learning_rate': 0.1, 'modelo__max_depth': 3, 'modelo__n_estimators': 100}


In [8]:
# para guardar los modelos entrenados usamos la siguiente librería
from joblib import dump
# guardamos los modelos entrenados
dump(PL1, "modelo_PL1.pkl")
dump(PL2_optimo, "modelo_PL2.pkl")
# guardamos la particion del df para predecir en otro archivo más limpio
x_train.to_pickle("x_train.pkl")
x_test.to_pickle("x_test.pkl")
y_train.to_pickle("y_train.pkl")
y_test.to_pickle("y_test.pkl")